**NOTE: This notebook is written for the Google Colab platform. However it can also be run (possibly with minor modifications) as a standard Jupyter notebook.** 



In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
from sklearn import datasets
from sklearn.cluster import MiniBatchKMeans, DBSCAN, AgglomerativeClustering
from scipy.spatial.distance import pdist, squareform
from collections import defaultdict
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML, SVG
from IPython.utils.capture import capture_output
from io import BytesIO

# get matplotblib's default color cycle
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }

def plot_clust(
    X, clusts, is_core=None, iclust=None, legend=True, ax=None,
    core_point_size=100, other_point_size=50, rasterized=False
):
    if ax is None:
        ax = plt.gca()

    if is_core is None:
        is_core = np.full(len(X), False)

    if iclust is None:
        iclust = np.max(clusts)

    index = clusts >= 0
    regular = np.where(index)[0]
    out_of_cluster = np.where(~index)[0]

    c = [colors[i] for i in clusts[regular]]
    s = [core_point_size if is_core[i] else other_point_size for i in regular]

    ax.scatter(X[regular, 0], X[regular, 1], c=c, s=s, rasterized=rasterized)
    ax.scatter(X[out_of_cluster, 0], X[out_of_cluster, 1], c='black',
        s=other_point_size, rasterized=rasterized)

    if legend:
        legend_patches = [mpatches.Patch(color=colors[i], label='Cluster {}'.format(i)) for i in range(iclust+1)]
        if len(out_of_cluster):
            legend_patches.append(mpatches.Patch(color='black', label='Out of cluster'))
        ax.legend(handles=legend_patches)

    ax.grid(ls='--')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

class DBScanAnimation:
    def __init__(
        self, X, min_pts=5, eps=0.5,
        fig=None, ax=None,
        highlight_pt=True, pt_circle=True,
        plot_callback=plot_clust,
        core_point_size=100,
        other_point_size=25,
        keep_ax_lims=True,
        **kwargs
    ):
        self.X = X
        self.min_pts = min_pts
        self.eps = eps

        self.highlight_pt = highlight_pt
        self.pt_circle = pt_circle
        self.core_point_size = core_point_size
        self.other_point_size = other_point_size

        self.keep_ax_lims = keep_ax_lims
        self.plot_callback = plot_callback
        self.kwargs = kwargs

        with capture_output() as io:
            if fig is None:
                self.fig = plt.figure()
            else:
                self.fig = fig

            if ax is None:
                self.ax = self.fig.gca()
            else:
                self.ax = ax

            plt.show()

        self.iclust = None
        self.is_core = None
        self.clusts = None
        self.clust_candidates = None
        self.clust_set = None
        self.cpt = None
        self.D = None
        self.lims = None

    def plot_func(
        self, cpt=None, cpt_size=None, cpt_color=None,
        circle_color='red', ax=None
    ):
        if ax is None:
            ax = self.ax

        if cpt_size is None:
            cpt_size = self.core_point_size

        if cpt_color is None:
            cpt_color = 'red'

        ax.clear()
        
        self.plot_callback(
            X=self.X, clusts=self.clusts, is_core=self.is_core,
            iclust=self.iclust, ax=ax,
            core_point_size=self.core_point_size,
            other_point_size=self.other_point_size,
            **self.kwargs
        )
        
        if self.highlight_pt and not cpt is None:
            ax.scatter(self.X[cpt, 0], self.X[cpt, 1], c=cpt_color, s=cpt_size)

            if self.pt_circle:
                circle = plt.Circle(
                    (X[cpt, 0], X[cpt, 1]),
                    self.eps, color=circle_color, fill=False, linewidth=3
                )

                ax.add_patch(circle)

        if self.keep_ax_lims and not self.lims is None:
            ax.set_xlim(self.lims[0])
            ax.set_ylim(self.lims[1])

    def __call__(self):
        self.iclust = -1
        self.clusts = np.full(len(self.X), -1)
        self.is_core = np.full(len(self.X), False)
        self.clust_candidates = set(range(len(self.X)))
        self.D = squareform(pdist(X))
        self.lims = None

        self.plot_func()
        self.lims = self.ax.get_xlim(), self.ax.get_ylim()
        yield

        self.iclust = 0

        while len(self.clust_candidates):
            self.cpt = self.clust_candidates.pop()
            nbrs = np.where((self.D[self.cpt] < self.eps) & (self.clusts == -1))[0]

            if len(nbrs) < self.min_pts:
                continue
            
            self.is_core[self.cpt] = True
            self.clusts[self.cpt] = self.iclust

            self.clusts[nbrs] = self.iclust
            self.clust_candidates.difference_update(nbrs)
            self.clust_set = set(nbrs)

            self.plot_func(self.cpt)
            yield

            while len(self.clust_set):
                self.cpt = self.clust_set.pop()
                self.clusts[self.cpt] = self.iclust

                neighbours_index = self.D[self.cpt] < self.eps
                not_in_clust_index = self.clusts == -1
                in_same_clust_index = self.clusts == self.iclust

                if (neighbours_index & (not_in_clust_index | in_same_clust_index)).sum() < self.min_pts:
                    continue

                nbrs = np.where(neighbours_index & not_in_clust_index)[0]

                self.is_core[self.cpt] = True
                self.clusts[nbrs] = self.iclust
                self.clust_candidates.difference_update(nbrs)    
                self.clust_set.update(nbrs)

                if len(nbrs):
                    self.plot_func(self.cpt)
                    yield

            self.iclust += 1

        self.iclust -= 1
        self.plot_func()
        yield

        return self.clusts, self.is_core

    def save_images(self, image_filename_tpl="output/dbscan_{step}.svg"):
        for istep, _ in enumerate(self()):
            self.fig.savefig(
                image_filename_tpl.format(step=istep),
                bbox_inches='tight'
            )
            
    def grab_frame(self, format='svg'):
        file = BytesIO()
        dbscan_ani.fig.savefig(fname=file, format=format)
        file.seek(0)
        image_str = file.read()
        return image_str
            
    def make_all_frames(self, format='svg'):
        gen = self()
        return [self.grab_frame(format=format) for _ in gen]

    def make_animation(self, interval=500):
        ani = FuncAnimation(
            self.fig, lambda x: [], frames=self,
            interval=interval, repeat=False
        )
        
        return ani

    def animate(self, interval=500):
        ani = self.make_animation(interval=interval)
        return self.show_animation(ani)
    
    def show_animation(self, ani):
        html = HTML(ani.to_jshtml())
        return html
    
# algos and hyperparams for the comparison of methods
algos = [
    ('kmeans', MiniBatchKMeans),
    ('hierarchical, ward', AgglomerativeClustering),
    ('hierarchical, single', AgglomerativeClustering),
    ('DBSCAN', DBSCAN)
]

hyperparams = defaultdict(dict)
hyperparams['kmeans']['circles'] = {'n_clusters': 2}
hyperparams['kmeans']['moons'] = {'n_clusters': 2}
hyperparams['kmeans']['blobs'] = {'n_clusters': 3}
hyperparams['kmeans']['varied_blobs'] = {'n_clusters': 3}

hyperparams['hierarchical, ward']['circles'] = {'n_clusters': 2, 'linkage': 'ward'}
hyperparams['hierarchical, ward']['moons'] = {'n_clusters': 2, 'linkage': 'ward'}
hyperparams['hierarchical, ward']['blobs'] = {'n_clusters': 3, 'linkage': 'ward'}
hyperparams['hierarchical, ward']['varied_blobs'] = {'distance_threshold': 75, 'n_clusters': None, 'linkage': 'ward'}

hyperparams['hierarchical, single']['circles'] = {'n_clusters': 2, 'linkage': 'single'}
hyperparams['hierarchical, single']['moons'] = {'n_clusters': 2, 'linkage': 'single'}
hyperparams['hierarchical, single']['blobs'] = {'n_clusters': 3, 'linkage': 'single'}
hyperparams['hierarchical, single']['varied_blobs'] = {'distance_threshold': 1.55, 'n_clusters': None, 'linkage': 'single'}

hyperparams['DBSCAN']['circles'] = {'eps': 0.15, 'min_samples': 5}
hyperparams['DBSCAN']['moons'] = {'eps': 0.18, 'min_samples': 5}
hyperparams['DBSCAN']['blobs'] = {'eps': 1.5, 'min_samples': 5}
hyperparams['DBSCAN']['varied_blobs'] = {'eps': 5.0, 'min_samples': 5}

## DBSCAN: Density-Based Clustering

DBSCAN [[dbscan1996]](#dbscan1996) is a well-known clustering method based on density. Unlike in $k$-means / hierachical clustering, you do not need to specify the number of clusters (explicitly or implicitly by specifying where to cut the dendrogram). The idea is as follows:

* If a point has at least $\textrm{min\_pts}$ neighbours at distances $\leq \epsilon$, it is a **core point**  of a cluster;


* Any other points within the distance of $\leq \epsilon$ are connected to it and also belong into the same cluster;


* If these connected points are:


* *Also core points* , then same applies to any points within the distance $\leq \epsilon$ from them, etc.;


* *Not core points* , then they are called **border points**  and no other points can connect to the cluster through them.



Using these principles, DBSCAN discovers clusters. Once DBSCAN is done, some isolated points, which are not connected to any cluster, might remain. These are simply marked as **out-of-cluster points**  – they are too far from any cluster to be counted among its points.

### A Demonstration

Now that we know about the basic principles, let us have a simple demonstration. We are going to create a synthetic dataset with three ball-shaped clusters.



In [ ]:
X, Y = datasets.make_blobs(
    n_samples=75,
    centers=[(1, 3), (3, 5), (5, 2)],
    cluster_std=0.4,
    random_state=0
)

plt.scatter(X[:, 0], X[:, 1], c='black')
plt.grid(ls='--')
plt.xlabel('x')
plt.ylabel('y')

Now we are going to go through what DBSCAN does step by step (using our custom class: `DBScanAnimation`). At the beginning, there will be no clusters.



In [ ]:
dbscan_ani = DBScanAnimation(X=X, eps=0.5, min_pts=5)
frames = dbscan_ani.make_all_frames()
display(SVG(frames[0]))

Then in the first step, we are going to pick points at random (or rather in an arbitrary order – we can pick them systematically if we like) until we arrive at one that is a core point (it has at least $\textrm{min\_pts}$ neighbours at distances $\leq \epsilon$). The figure highlighs one such point in red along with its $\epsilon$-neighbourhood.

This point is used to start the first cluster. The cluster will initially contain the core point and its $\epsilon$-neigbourhood (the points are colour-coded as cluster 0 in the figure).



In [ ]:
display(SVG(frames[1]))

As the next step we go through all these points, determine which of them are core points (we assign larger circles to those in the plot) and also add their $\epsilon$-neighbours into the cluster, etc. Points that belong into the cluster and they are not core points, are called border points. These still belong into the cluster, but further points cannot connect to the cluster through them.

Let's show the first few steps of the process here.



In [ ]:
display(SVG(frames[2]))
display(SVG(frames[3]))

display(HTML("<center><strong>⋮</strong></center>"))

display(SVG(frames[7]))

Once we have exhausted all points connected to the first cluster, we go on with the remaining out-of-cluster points, trying to idenfity a further core point that we could use to start the next cluster. After that, we just keep going on analogically.



In [ ]:
display(SVG(frames[8]))

Eventually we will have gone through all available points. It may happen that at that step some points will still be **out of cluster** . This is fine – it means that they do not meet the criteria to belong into a cluster. In practical applications it is often going to be useful to be able to identify such points.



In [ ]:
display(SVG(frames[-1]))

Finally, let's check one of the border points. We should see that while it is in the $\epsilon$-neighbourhood of a core point, it does not have enough points in its own $\epsilon$-neighbourhood to be itself a core point.



In [ ]:
ax = plt.gca()
boundary_point = np.where(dbscan_ani.clusts != -1 & ~dbscan_ani.is_core)[0][0]
dbscan_ani.plot_func(cpt=boundary_point, ax=ax, cpt_size=30)

Finally, we can generate an animated version of the entire procedure so that you can go through all the steps at will.



In [ ]:
dbscan_ani = DBScanAnimation(X=X, eps=0.5, min_pts=5)
dbscan_ani.animate()

### DBSCAN in Scikit-Learn

Now that we understand the basics of how DBSCAN works, let us see how it can be applied in practice using `scikit-learn`. There are no surprises – DBSCAN uses the standard `scikit-learn` interface. The "cluster number" of out-of-cluster points is `-1`. The numbering of the regular clusters starts from 0.

Note that in general, it will be a good idea to **standardize the data**  before applying DBSCAN (just as we did with other clustering methods). We only omit standardization here because these are purely illustrative examples and the data does not really require standardization.



In [ ]:
model = DBSCAN(eps=0.5, min_samples=5)
clusts = model.fit_predict(X)

In [ ]:
plot_clust(X=X, clusts=clusts)

### Clustering Methods: A Comparison

Finally, we are going to include a small comparison of $k$-means, hierarchical clustering (with ward and single linkage) and DBSCAN. We are going to be using several toy datasets to better showcase what kinds of clusters each method is going to be able to discover. Let's first generate the data and plot it: we are going to use data from an [example in scikit-learn's documentation](https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html).



In [ ]:
#@title -- Data generation code -- { display-mode: "form" }
n_samples = 1500
circles, _ = datasets.make_circles(n_samples=n_samples, factor=0.5, noise=0.05, random_state=0)
moons, _ = datasets.make_moons(n_samples=n_samples, noise=0.05, random_state=0)
blobs, _ = datasets.make_blobs(n_samples=n_samples, random_state=8)
varied_blobs, _ = datasets.make_blobs(n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=170)

data = [
    ('circles', circles),
    ('moons', moons),
    ('blobs', blobs),
    ('varied_blobs', varied_blobs)
]

# plotting
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(8, 8))

axes[0, 0].scatter(circles[:, 0], circles[:, 1])
axes[0, 0].set_title('circles')
axes[0, 0].grid(ls='--')
axes[0, 0].set_ylabel('y')

axes[0, 1].scatter(moons[:, 0], moons[:, 1])
axes[0, 1].set_title('moons')
axes[0, 1].grid(ls='--')

axes[1, 0].scatter(blobs[:, 0], blobs[:, 1])
axes[1, 0].set_title('blobs')
axes[1, 0].grid(ls='--')
axes[1, 0].set_xlabel('x')
axes[1, 0].set_ylabel('y')

axes[1, 1].scatter(varied_blobs[:, 0], varied_blobs[:, 1])
axes[1, 1].set_title('varied blobs')
axes[1, 1].grid(ls='--')
axes[1, 1].set_xlabel('x')

Now we are going to apply each of the clustering methods to each dataset (with slightly different hyperparameters for each – but let's not worry about that here) and display the results in a plot.



In [ ]:
#@title -- Clustering code -- { display-mode: "form" }
fig, axes = plt.subplots(len(data), len(algos))
fig.set_size_inches(10, 8)

for jax in range(axes.shape[1]):
    axes[0][jax].set_title(algos[jax][0])

for iax in range(axes.shape[0]):
    for jax in range(axes.shape[1]):
        ax = axes[iax][jax]
        X = data[iax][1]
        alg = algos[jax][1]
        params = hyperparams[algos[jax][0]][data[iax][0]]
        
        model = alg(**params)
        clusts = model.fit_predict(X)
        plot_clust(X=X, clusts=clusts, ax=ax, legend=False,
            other_point_size=5, rasterized=True)
        
        if jax > 0:
            ax.set_ylabel('')
            
        if iax < axes.shape[0] - 1:
            ax.set_xlabel('')

#### $k$-means

One thing we will be able to see immediately is that $k$-means will only really be able to deal correctly with ball-shaped clusters – this is why it is only really going to work well with the blobs dataset.

#### Hierarchical Clustering

With hierarchical clustering, we can change the linkage method (how distances between clusters are determined). Note that ward linkage (and other linkages such as average or complete) do not work well with circles and moons. Single linkage is looking at the smallest distance between clusters so it is more locally oriented and works well in those cases. However, it does not really work well in the case of variable-density blobs.

#### DBSCAN

Similarly, DBSCAN works great for blobs, circles and moons, but – being density-based – **it cannot handle clusters with significantly different densities – there is no good value for $\epsilon$ because it will either be too small for one cluster or too large for another.** 

To make the problem clearer, let's run DBSCAN on the varied blobs dataset again with two different values of $\epsilon$.



In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))

model = DBSCAN(eps=5, min_samples=5)
clusts = model.fit_predict(varied_blobs)
plot_clust(X=varied_blobs, clusts=clusts, ax=axes[0],
    legend=False, other_point_size=10, rasterized=True)

model = DBSCAN(eps=0.5, min_samples=5)
clusts = model.fit_predict(varied_blobs)
plot_clust(X=varied_blobs, clusts=clusts, ax=axes[1],
    legend=False, other_point_size=10, rasterized=True)

### References

<a id="dbscan1996">[dbscan1996]</a> Ester, M., Kriegel, H.P., Sander, J. and Xu, X., 1996, August. A density-based algorithm for discovering clusters in large spatial databases with noise. In kdd (Vol. 96, No. 34, pp. 226-231).

